In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizerFast, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model = BertModel.from_pretrained("kykim/bert-kor-base")

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
data = pd.read_csv("./data/BERT_sim_sents1.csv")
data

,문장 A,문장 B
0,간은 피로회복에 중요하다.,신장은 노폐물 배출에 중요하다.
1,요가를 하면 마음이 안정된다.,스트레칭을 하면 유연해진다.
2,간을 맞으면 아프다.,턱을 맞으면 기절한다.
3,사과는 피부 미용에 좋다.,녹용은 간에 좋다.
4,미나리는 피를 맑게한다.,인삼은 몸에 에너지를 충전한다.


# BERT 모델을 사용해서 문장의 유사도 확인
현재 bert의 per-train의 output을 mean_pooling을 이용해서 sent vector로 바꾼 후 유사도를 계산하는 방식

In [107]:
# 하나의 문장씩 비교 하는 방법
def word_sim(sent1 , sent2):
    words = [sent1, sent2]
    tokens = {'input_ids' : [] , 'attention_mask' : []}

    
    for word in words:
        print(word)
        print("token의 수 : " +str(len(tokenizer.encode_plus(word)['input_ids'])))
        new_tokens = tokenizer.encode_plus(word , max_length=30, truncation= True , padding='max_length' , return_tensors='pt')
        
        tokens['input_ids'].append(new_tokens['input_ids'][0])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0])
    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state
    #return embeddings
    attention_mask = tokens['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask
    summed = torch.sum(masked_embeddings,1)
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed / summed_mask
    mean_pooled = mean_pooled.detach().numpy()
    return cosine_similarity( mean_pooled[0].reshape(1,-1) , mean_pooled[1].reshape(1,-1))[0][0]

In [110]:
for idx in range(0,5):
    sim = word_sim(data['문장 A'][idx],data['문장 B'][idx])
    print(f'{idx}번째의 문장의 유사도는 {sim}\n')

간은 피로회복에 중요하다.
token의 수 : 9
신장은 노폐물 배출에 중요하다.
token의 수 : 9
0번째의 문장의 유사도는 0.8338543176651001

요가를 하면 마음이 안정된다.
token의 수 : 9
스트레칭을 하면 유연해진다.
token의 수 : 8
1번째의 문장의 유사도는 0.7455722689628601

간을 맞으면 아프다.
token의 수 : 6
턱을 맞으면 기절한다.
token의 수 : 8
2번째의 문장의 유사도는 0.6222368478775024

사과는 피부 미용에 좋다.
token의 수 : 9
녹용은 간에 좋다.
token의 수 : 7
3번째의 문장의 유사도는 0.7551013231277466

미나리는 피를 맑게한다.
token의 수 : 8
인삼은 몸에 에너지를 충전한다.
token의 수 : 9
4번째의 문장의 유사도는 0.6823358535766602



실험에 사용된 PC의 환경(연구실 컴퓨터)
```
프로세서	Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz   3.40 GHz
설치된 RAM	16.0GB(15.9GB 사용 가능)
```

# Tokenizer에서 실행 시간 확인
tokenizer에서는 대략 1000 nano sec 시간 소요

In [122]:
def token_time(sent):
    tokenizer.encode_plus(sent , max_length=10, truncation= True , padding='max_length' , return_tensors='pt')


In [130]:
%%time
token_time(data['문장 A'][0])

Wall time: 997 µs


## 두 문장의 유사도 실행 시간

대략 125 ms 소요

In [146]:
%%time
word_sim(data['문장 A'][0],data['문장 B'][1])

간은 피로회복에 중요하다.
token의 수 : 9
스트레칭을 하면 유연해진다.
token의 수 : 8
Wall time: 124 ms


0.5451042

### 위 수행 시간 비교를 10번 반복 했을 때의 실행시간

In [148]:
%%time
for _ in range(0,9):
    word_sim(data['문장 A'][0],data['문장 A'][0])

간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
간은 피로회복에 중요하다.
token의 수 : 9
Wall time: 977 ms


## 모든 문장을 한번에 입력 하는 경우를 Test

0.52s정도 나옴

In [175]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [179]:
%%time
# senst = ['상품을 표현하는 문장' , '에너지에 대한 문장' , '회복에 대한 문장' , '순환에 대한 문장' , '정화에 대한 문장']
sents = ['차원을 설명하는 문장', data['문장 A'][1] , data['문장 A'][2] , data['문장 A'][3] , data['문장 A'][4]]
add_sents = sents + sents + sents + sents + sents + sents + sents + sents + sents + sents
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in add_sents:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=10,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
attention_mask = tokens['attention_mask']
outputs = model(**tokens)
embeddings = outputs.last_hidden_state
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
masked_embeddings = embeddings * mask
summed = torch.sum(masked_embeddings, 1)
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
mean_pooled = summed / summed_mask
mean_pooled = mean_pooled.detach().numpy()


for idx in range(0,len(add_sents),5):
    print(cosine_similarity([mean_pooled[idx]],mean_pooled[idx+1:idx+5]))



[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
[[0.45982772 0.3141464  0.37281248 0.35410008]]
Wall time: 515 ms
